# Investment Tracker

## Import das Libs

In [108]:
import os
import pandas as pd

## Transactions

In [109]:
class Transactions:
    
    def __init__(self):
        self.data_raw = None
        self.data = None
    
    
    @staticmethod
    def rename_cols(dataframe):
        dataframe = dataframe.rename(columns={'Entrada/Saída': "FLOW",
                                              'Data': 'DATE',
                                              'Movimentação': 'ACTION_TYPE',
                                              'Produto': "PRODUCT_NAME",
                                              'Instituição': 'BROKER',
                                              'Quantidade': 'QUANTITY',
                                              'Preço unitário': 'PRICE_UNIT',
                                              'Valor da Operação': "VALUE",
                                              'Sigla': 'TICKER',
                                              'Tipo': 'ASSET_TYPE'})
        return dataframe
    
    
    @staticmethod
    def find_ticker(product_name):
    
        ticker = product_name.split(" - ")[0].strip()

        return ticker
    
    
    @staticmethod
    def adjust_product_name(product_name):
    
        product_name = " ".join(product_name.split())
        components = product_name.split(" - ")
        product_name = " ".join(components[1:])

        return product_name
    
    
    @staticmethod
    def type_of_product(ticker):
    
        if any(x in ticker for x in ["33", "34"]):
            return "BDR"
        elif any(x in ticker for x in ["11", "12"]):
            return "FII"
        elif any(x in ticker for x in ["1", "2", "3", "4"]):
            return "ACAO"
        else:
            return 'OTHER'
    
    
    @staticmethod
    def adjust_flow(flow):
    
        if flow.strip() == "Credito":
            return "Entrada"
        else:
            return "Saida"   

    
    def preprocessing(self, log=False):
        
        data_preprocessed = self.rename_cols(self.data_raw)
        data_preprocessed.sort_values(by="DATE", inplace=True)
        data_preprocessed.reset_index(drop=True, inplace=True)
        data_preprocessed['TICKER'] = data_preprocessed.PRODUCT_NAME.apply(self.find_ticker)
        data_preprocessed['PRODUCT_NAME'] = data_preprocessed.PRODUCT_NAME.apply(self.adjust_product_name)
        data_preprocessed['ASSET_TYPE'] = data_preprocessed.TICKER.apply(self.type_of_product)
        data_preprocessed['FLOW'] = data_preprocessed['FLOW'].apply(self.adjust_flow)
        data_preprocessed = data_preprocessed[["FLOW", "DATE", "ACTION_TYPE", "ASSET_TYPE", "TICKER", "PRODUCT_NAME", "BROKER", "QUANTITY", "PRICE_UNIT", "VALUE"]]
        
        if log == True:
            print("DONE: Transaction preprocessing")
        
        self.data = data_preprocessed
            

    def read_database(self, log=False):
        path = os.getcwd()
        files = os.listdir(path + "/databases")
    
        files_excel = [file for file in files if file[-4:] == 'xlsx']
        files_excel
    
        if log == True:
            [print(f) for f in files if f[-4:] == 'xlsx']
            
        database = pd.DataFrame()
        for file in files_excel:
            data = pd.read_excel('databases/' + file,
                                 parse_dates=['Data'],
                                 date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
            database = database.append(data)
            
        self.data_raw = database
        

In [142]:
transactions = Transactions()
transactions.read_database(log=True)

movimentacao-2019.xlsx
movimentacao-2020.xlsx
movimentacao-2021.xlsx
movimentacao-2022.xlsx


c:\Users\joaoa\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\joaoa\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [143]:
transactions.preprocessing(log=True)

DONE: Transaction preprocessing


In [144]:
transactions.data_raw.head()

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
0,Credito,2019-12-31,Transferência - Liquidação,BIDI4 - BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Credito,2019-12-31,Transferência - Liquidação,MGLU3 - MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Credito,2019-12-31,Transferência - Liquidação,VVAR3 - VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Credito,2019-12-31,Transferência - Liquidação,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
0,Credito,2020-12-29,Atualização,VVAR3 - VIA VAREJO S.A.,NU INVEST CORRETORA DE VALORES S.A.,249,-,-


In [113]:
transactions.data.head()

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
0,Entrada,2019-12-31,Transferência - Liquidação,ACAO,BIDI4,BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Entrada,2019-12-31,Transferência - Liquidação,ACAO,MGLU3,MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Entrada,2019-12-31,Transferência - Liquidação,ACAO,VVAR3,VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Entrada,2019-12-31,Transferência - Liquidação,FII,SPTW11,SP DOWNTOWN FDO INV IMOB FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Saida,2020-01-02,Juros Sobre Capital Próprio - Transferido,ACAO,BIDI4,BANCO INTER S/A,NU INVEST CORRETORA DE VALORES S.A.,104,0.02,1.6


### Old

Import das Bases

In [114]:
path = os.getcwd()
files = os.listdir(path + "/bases")
[print(f) for f in files if f[-4:] == 'xlsx'];

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'c:\\Users\\joaoa\\Documents\\GitHub\\investment-tracker/bases'

In [ ]:
files_excel = [f for f in files if f[-4:] == 'xlsx']
files_excel

NameError: name 'files' is not defined

In [ ]:
df = pd.DataFrame()

for file in files_excel:
    data = pd.read_excel('bases/' + file,
                         parse_dates=['Data'],
                         date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
    df = df.append(data)

NameError: name 'files_excel' is not defined

In [ ]:
df[:20]

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
0,Credito,2019-12-31,Transferência - Liquidação,BIDI4 - BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Credito,2019-12-31,Transferência - Liquidação,MGLU3 - MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Credito,2019-12-31,Transferência - Liquidação,VVAR3 - VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Credito,2019-12-31,Transferência - Liquidação,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
0,Credito,2020-12-29,Atualização,VVAR3 - VIA VAREJO S.A. ...,EASYNVEST - TITULO CV S/A,249,-,-
1,Credito,2020-12-17,Dividendo,CYRE3 - CYRELA BRAZIL REALTY S.A.EMPREEND E PA...,EASYNVEST - TITULO CV S/A,70,0.26,17.98
2,Credito,2020-12-15,Dividendo,PETR3 - PETROLEO BRASILEIRO S.A. PETROBRAS ...,EASYNVEST - TITULO CV S/A,100,0.23,23.36
3,Credito,2020-12-15,Rendimento,PETR3 - PETROLEO BRASILEIRO S.A. PETROBRAS ...,EASYNVEST - TITULO CV S/A,100,0.01,0.5
4,Credito,2020-12-08,Amortização,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII ...,EASYNVEST - TITULO CV S/A,20,0.54,10.8
5,Credito,2020-12-08,Rendimento,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII ...,EASYNVEST - TITULO CV S/A,20,0.7,14


In [ ]:
df.sort_values(by="Data", inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
0,Credito,2019-12-31,Transferência - Liquidação,BIDI4 - BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Credito,2019-12-31,Transferência - Liquidação,MGLU3 - MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Credito,2019-12-31,Transferência - Liquidação,VVAR3 - VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Credito,2019-12-31,Transferência - Liquidação,SPTW11 - SP DOWNTOWN FDO INV IMOB - FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Debito,2020-01-02,Juros Sobre Capital Próprio - Transferido,BIDI4 - BANCO INTER S.A. ...,EASYNVEST - TITULO CV S/A,104,0.02,1.6
...,...,...,...,...,...,...,...,...
258,Credito,2022-05-16,Dividendo,PETR4 - PETROLEO BRASILEIRO S/A PETROBRAS,NU INVEST CORRETORA DE VALORES S.A.,86,2.86,246.05
259,Credito,2022-05-16,Rendimento,PETR3 - PETROLEO BRASILEIRO S/A PETROBRAS,NU INVEST CORRETORA DE VALORES S.A.,100,0.11,8.46
260,Credito,2022-05-16,Dividendo,PETR3 - PETROLEO BRASILEIRO S/A PETROBRAS,NU INVEST CORRETORA DE VALORES S.A.,100,2.86,286.1
261,Credito,2022-05-25,Rendimento,BTLG11 - BTG PACTUAL LOGISTICA FUNDO DE ...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52


In [ ]:
def find_ticker(product_name):
    
    ticker = product_name.split(" - ")[0].strip()
    
    return ticker

In [ ]:
df.Produto.iloc[21]

'XPML11 - XP MALLS FDO INV IMOB FII                         '

In [ ]:
def adjust_product_name(product_name):
    
    product_name = " ".join(product_name.split())
    components = product_name.split(" - ")
    
    product_name = " ".join(components[1:])
    
    return product_name

In [ ]:
adjust_product_name("PETR3        - PETROLEO BRASILEIRO S/A PETROBRAS")

'PETROLEO BRASILEIRO S/A PETROBRAS'

In [ ]:
df['Sigla'] = df.Produto.apply(find_ticker)

In [ ]:
df['Produto'] = df.Produto.apply(adjust_product_name)

In [ ]:
df.head()

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Sigla
0,Credito,2019-12-31,Transferência - Liquidação,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.7,1632.8,BIDI4
1,Credito,2019-12-31,Transferência - Liquidação,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21,MGLU3
2,Credito,2019-12-31,Transferência - Liquidação,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.9,VVAR3
3,Credito,2019-12-31,Transferência - Liquidação,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98,SPTW11
4,Debito,2020-01-02,Juros Sobre Capital Próprio - Transferido,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.6,BIDI4


In [ ]:
df.Sigla.unique()

array(['BIDI4', 'MGLU3', 'VVAR3', 'SPTW11', 'XPML11', 'BTOW3', 'JHSF3',
       'AZUL4', 'PETR3', 'WEGE3', 'ITUB4', 'CYRE3', 'BIDI2', 'BTOW1',
       'B3SA3', 'VALE3', 'SULA11', 'BBDC3', 'TSLA34', 'CSAN3', 'ABEV3',
       'KISU11', 'NVDC34', 'AMZO34', 'XINA11', 'PETR4', 'S1PO34',
       'KISU12', 'BTLG11', 'BPAC3', 'BTLG12', 'NUBR33', 'NFLX34'],
      dtype=object)

In [ ]:
def type_of_product(ticker):
    
    if any(x in ticker for x in ["33", "34"]):
        return "BDR"
    elif any(x in ticker for x in ["11", "12"]):
        return "FII"
    elif any(x in ticker for x in ["1", "2", "3", "4"]):
        return "ACAO"
    else:
        return 'OTHER'

In [ ]:
df['Tipo'] = df.Sigla.apply(type_of_product)

In [ ]:
df.head()

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Sigla,Tipo
0,Credito,2019-12-31,Transferência - Liquidação,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.7,1632.8,BIDI4,ACAO
1,Credito,2019-12-31,Transferência - Liquidação,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21,MGLU3,ACAO
2,Credito,2019-12-31,Transferência - Liquidação,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.9,VVAR3,ACAO
3,Credito,2019-12-31,Transferência - Liquidação,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98,SPTW11,FII
4,Debito,2020-01-02,Juros Sobre Capital Próprio - Transferido,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.6,BIDI4,ACAO


In [ ]:
df['Data'] = pd.to_datetime(df.Data, format="%d-%m-%Y")

In [ ]:
df[df['Sigla'] == "CYRE3"].sort_values(by="Data")

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Sigla,Tipo
56,Credito,2020-06-03,Transferência - Liquidação,CYRELA BRAZIL REALTY S.A.EMPREEND E PART (CYRE3),EASYNVEST - TITULO CV S/A,50,17.18,859,CYRE3,ACAO
63,Credito,2020-07-15,Transferência - Liquidação,CYRELA BRAZIL REALTY S.A.EMPREEND E PART (CYRE3),EASYNVEST - TITULO CV S/A,20,27.57,551.4,CYRE3,ACAO
68,Credito,2020-07-28,Transferência - Liquidação,CYRELA BRAZIL REALTY S.A.EMPREEND E PART (CYRE3),EASYNVEST - TITULO CV S/A,30,25.26,757.8,CYRE3,ACAO
113,Debito,2020-11-04,Transferência - Liquidação,CYRELA BRAZIL REALTY S/A EMPREENDIMENTOS E PAR...,NU INVEST CORRETORA DE VALORES S.A.,100,22.74,2274,CYRE3,ACAO
124,Credito,2020-12-17,Dividendo,CYRELA BRAZIL REALTY S.A.EMPREEND E PART (CYRE3),EASYNVEST - TITULO CV S/A,70,0.26,17.98,CYRE3,ACAO
128,Credito,2021-01-07,Transferência - Liquidação,CYRELA BRAZIL REALTY S/A EMPREENDIMENTOS E PAR...,NU INVEST CORRETORA DE VALORES S.A.,100,27.86,2786,CYRE3,ACAO
184,Credito,2021-06-14,Dividendo,CYRELA BRAZIL REALTY S/A EMPREENDIMENTOS E PAR...,EASYNVEST - TITULO CV S/A,100,1.09,108.72,CYRE3,ACAO
220,Debito,2021-11-04,Transferência - Liquidação,CYRELA BRAZIL REALTY S/A EMPREENDIMENTOS E PAR...,NU INVEST CORRETORA DE VALORES S.A.,100,14.53,1453,CYRE3,ACAO


In [ ]:
df['Entrada/Saída'].unique()

array(['Credito', 'Debito'], dtype=object)

In [ ]:
def adjust_entrada_saida(entrada_saida):
    
    if entrada_saida.strip() == "Credito":
        return "Entrada"
    else:
        return "Saida"

In [ ]:
df['Fluxo'] = df['Entrada/Saída'].apply(adjust_entrada_saida)

In [ ]:
df

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Sigla,Tipo,Fluxo
0,Credito,2019-12-31,Transferência - Liquidação,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.7,1632.8,BIDI4,ACAO,Entrada
1,Credito,2019-12-31,Transferência - Liquidação,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21,MGLU3,ACAO,Entrada
2,Credito,2019-12-31,Transferência - Liquidação,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.9,VVAR3,ACAO,Entrada
3,Credito,2019-12-31,Transferência - Liquidação,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98,SPTW11,FII,Entrada
4,Debito,2020-01-02,Juros Sobre Capital Próprio - Transferido,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.6,BIDI4,ACAO,Saida
...,...,...,...,...,...,...,...,...,...,...,...
258,Credito,2022-05-16,Dividendo,PETROLEO BRASILEIRO S/A PETROBRAS (PETR4),NU INVEST CORRETORA DE VALORES S.A.,86,2.86,246.05,PETR4,ACAO,Entrada
259,Credito,2022-05-16,Rendimento,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,0.11,8.46,PETR3,ACAO,Entrada
260,Credito,2022-05-16,Dividendo,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,2.86,286.1,PETR3,ACAO,Entrada
261,Credito,2022-05-25,Rendimento,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52,BTLG11,FII,Entrada


In [ ]:
df = df[['Fluxo', "Data", "Movimentação", "Tipo", "Sigla", "Produto", "Instituição", "Quantidade", "Preço unitário", "Valor da Operação"]]

In [ ]:
df.columns

Index(['Fluxo', 'Data', 'Movimentação', 'Tipo', 'Sigla', 'Produto',
       'Instituição', 'Quantidade', 'Preço unitário', 'Valor da Operação'],
      dtype='object')

In [ ]:
 df.rename(columns={'Fluxo': "FLOW",
                    'Data': 'DATE',
                    'Movimentação': 'ACTION_TYPE',
                    'Produto': "PRODUCT_NAME",
                    'Instituição': 'BROKER',
                    'Quantidade': 'QUANTITY',
                    'Preço unitário': 'PRICE_UNIT',
                    'Valor da Operação': "VALUE",
                    'Sigla': 'TICKER',
                    'Tipo': 'ASSET_TYPE'}, inplace=True)

C:\Users\joaoa\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
df.DATE.dtype

dtype('<M8[ns]')

In [ ]:
df.head()

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
0,Entrada,2019-12-31,Transferência - Liquidação,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Entrada,2019-12-31,Transferência - Liquidação,ACAO,MGLU3,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Entrada,2019-12-31,Transferência - Liquidação,ACAO,VVAR3,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Entrada,2019-12-31,Transferência - Liquidação,FII,SPTW11,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Saida,2020-01-02,Juros Sobre Capital Próprio - Transferido,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.6


In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'])

C:\Users\joaoa\AppData\Local\Temp/ipykernel_20296/2541906660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE'] = pd.to_datetime(df['DATE'])


In [ ]:
print(type(df['DATE'].iloc[0])) 

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
df.sort_values(by=['DATE', 'FLOW'], ascending=[True, False], inplace=True)

C:\Users\joaoa\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
print(type(df['PRICE_UNIT'].iloc[0])) 
print(type(df['VALUE'].iloc[0])) 

<class 'float'>
<class 'float'>


In [ ]:
df['PRICE_UNIT'] = pd.to_numeric(df['PRICE_UNIT'], errors='coerce')
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce');

C:\Users\joaoa\AppData\Local\Temp/ipykernel_20296/3741329072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PRICE_UNIT'] = pd.to_numeric(df['PRICE_UNIT'], errors='coerce')
C:\Users\joaoa\AppData\Local\Temp/ipykernel_20296/3741329072.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce');


In [ ]:
df

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
0,Entrada,2019-12-31,Transferência - Liquidação,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,15.70,1632.80
1,Entrada,2019-12-31,Transferência - Liquidação,ACAO,MGLU3,MAGAZINE LUIZA S.A. (MGLU3),EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Entrada,2019-12-31,Transferência - Liquidação,ACAO,VVAR3,VIA VAREJO S.A. (VVAR3),EASYNVEST - TITULO CV S/A,130,11.73,1524.90
3,Entrada,2019-12-31,Transferência - Liquidação,FII,SPTW11,SP DOWNTOWN FDO INV IMOB FII (SPTW11),EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Saida,2020-01-02,Juros Sobre Capital Próprio - Transferido,ACAO,BIDI4,BANCO INTER S.A. (BIDI4),EASYNVEST - TITULO CV S/A,104,0.02,1.60
...,...,...,...,...,...,...,...,...,...,...
258,Entrada,2022-05-16,Dividendo,ACAO,PETR4,PETROLEO BRASILEIRO S/A PETROBRAS (PETR4),NU INVEST CORRETORA DE VALORES S.A.,86,2.86,246.05
259,Entrada,2022-05-16,Rendimento,ACAO,PETR3,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,0.11,8.46
260,Entrada,2022-05-16,Dividendo,ACAO,PETR3,PETROLEO BRASILEIRO S/A PETROBRAS (PETR3),NU INVEST CORRETORA DE VALORES S.A.,100,2.86,286.10
261,Entrada,2022-05-25,Rendimento,FII,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,NU INVEST CORRETORA DE VALORES S.A.,16,0.72,11.52


In [ ]:
df.ACTION_TYPE.unique()

array(['Transferência - Liquidação',
       'Juros Sobre Capital Próprio - Transferido',
       'Juros Sobre Capital Próprio', 'Amortização', 'Rendimento',
       'Transferência', 'Direito de Subscrição', 'Dividendo',
       'Direitos de Subscrição - Não Exercido',
       'Cessão de Direitos - Solicitada', 'Cessão de Direitos',
       'Desdobro', 'Atualização'], dtype=object)

## Asset

In [ ]:
class Asset:
    
    def __init__(self, ticker, product_name, asset_type):
        self.ticker = ticker
        self.product_name = product_name
        self.asset_type = asset_type
        self.quantity = 0
        self.value = 0 
        self.price_unit = 0
        #self.transaction_history = dataframe
        #self.dividendo_history = dataframe
        self.income = 0
        
    def atualiza_price_unit(self):
        
        if self.quantity == 0:
            return None
        
        else:
            self.price_unit = self.value / self.quantity
            return self.value / self.quantity
        

    def set_quantity(self, quantidade):
        self.quantity = quantidade

        
    def set_value(self, valor):
        self.value = valor
        
    def set_income(self, income_value):
        self.income = income_value
        
        
    def buy(self, quantidade_compra, valor_compra):
        
        quantidade_atual = self.quantity 
        quantidade_final = quantidade_atual + quantidade_compra
        
        valor_atual = self.value 
        valor_final = valor_atual + valor_compra
        
        self.set_quantity(quantidade_final)
        self.set_value(valor_final)
        self.atualiza_price_unit()
        
        
    def sell(self, quantidade_venda, valor_venda):
        
        quantidade_atual = self.quantity 
        quantidade_final = quantidade_atual - quantidade_venda
        
#         valor_atual = self.value 
#         valor_final = valor_atual - valor_venda
        
        if quantidade_final < 0:
            raise Exception('Você tentou vendar uma quantidade maior do que possui para o ativo:', self.ticker)
            
#         if valor_final < 0:
#             raise Exception('Você tentou vendar um valor maior do que possui para o ativo:', self.ticker)    
        
        
        self.set_quantity(quantidade_final)
#         self.set_value(valor_final)
        self.atualiza_price_unit()    
    
    
    
    def split(self, quantidade_split, method='add'):
        
        if method == 'add':
            quantidade_atual = self.quantity 
            quantidade_final = quantidade_atual + quantidade_split
            
        elif method == 'split':
            quantidade_atual = self.quantity
            quantidade_final = quantidade_atual * quantidade_split
            
        self.set_quantity(quantidade_final)
        self.atualiza_price_unit()
        
        
    def income_distribution(self, income_received):
        
        total_asset_income = self.income + income_received
        self.set_income(total_asset_income)
        

        

    def imprime_ativo(self):
        
        print("Ticker:", self.ticker)
        print("Nome do produto:", self.product_name)
        print("Tipo:", self.asset_type)
        print("Quantidade:", self.quantity)
        print("Valor:", self.value)
        print("Preço médio: {:.2f}".format(self.price_unit))
        print("Rendimentos: {:.2f}".format(self.income))
    

### Testing - Asset class

In [ ]:
# Cria um exemplo de ativo
teste = Asset("MGLU3", "MAGAZINE LUIZA S.A. (MGLU3)", "ACAO")

In [ ]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00


In [ ]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00


In [ ]:
teste.set_quantity(200)

In [ ]:
teste.set_value(400)

In [ ]:
teste.atualiza_price_unit()

2.0

In [ ]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 200
Valor: 400
Preço médio: 2.00
Rendimentos: 0.00


In [ ]:
teste.buy(100, 50)

In [ ]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 300
Valor: 450
Preço médio: 1.50
Rendimentos: 0.00


In [ ]:
teste.sell(200, 150)

In [ ]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 100
Valor: 450
Preço médio: 4.50
Rendimentos: 0.00


In [ ]:
teste.split(400)

In [ ]:
teste.imprime_ativo()

Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 500
Valor: 450
Preço médio: 0.90
Rendimentos: 0.00


## Transaction

In [115]:
class Transaction:
    
    def __init__(self, flow, date, action, asset_type, asset_ticker, asset_name, broker, quantity, price, value):
        self.flow = flow
        self.date = date
        self.action = action
        self.asset_type = asset_type
        self.asset_ticker = asset_ticker
        self.asset_name = asset_name
        self.broker = broker
        self.quantity = quantity
        self.price = price
        self.value = value

    def print_transaction(self):
        print(
            self.flow.ljust(10),
            str(self.date).ljust(22),
            self.asset_ticker.ljust(10),
            str(self.price).ljust(8),
            str(self.quantity).ljust(22)
        )
        

## Wallet

In [182]:
class Wallet:

    def __init__(self, name):
        self.name = name
        self.asset_list = []

        
    def asset_exist(self, asset_ticker_searched):      
        for asset in self.asset_list:
            if asset.ticker == asset_ticker_searched:
                return True
        return False
            
        
    def add_asset(self, ticker, product_name, asset_type, log=False):
        
        if self.asset_exist(ticker):
            if log:
                print("{} alredy exist".format(ticker))
            pass
           
        else:
            created_asset = Asset(ticker, product_name, asset_type)
            self.asset_list.append(created_asset)        
            if log:
                print("{} added".format(ticker))
          
        
    def find_asset(self, find_ticker):
        for asset in self.asset_list:
            if asset.ticker == find_ticker:
                return asset
                break            
        

    def print_wallet(self):        
        wallet_position = pd.DataFrame(columns=['TICKER', 'TYPE', 'PRODUCT_NAME', 'QUANTITY', 'PRICE', "VALUE", "INCOME"])

        for asset in self.asset_list:
            asset_info = [asset.ticker, asset.asset_type, asset.product_name, asset.quantity, round(asset.price_unit,2), asset.value, asset.income]

            wallet_position.loc[len(wallet_position)] = asset_info

        return wallet_position
     
    
    def count_assets(self):
        return len(self.asset_list)

    

            
    def track(self, transactions, log_process=False, log_transaction=False):

        '''
        Percorre o banco de dados de transações, atualizando os ativos na wallet
        '''

        for index, row in transactions.data.iterrows():

            # Mapeia os atributos do banco no objeto transaction
            transaction = Transaction(
                flow = row['FLOW'],
                date = row['DATE'],
                action = row['ACTION_TYPE'],
                asset_type = row['ASSET_TYPE'],
                asset_ticker = row['TICKER'],
                asset_name = row['PRODUCT_NAME'], 
                broker = row['BROKER'],
                quantity = row['QUANTITY'],
                price = row['PRICE_UNIT'],
                value = row['VALUE']
                )

            if log_transaction:
                transaction.print_transaction()

            
            if transaction.action == "Transferência - Liquidação":
                
                # Avalia se o ativo existe, se não cria na wallet
                if not self.asset_exist(transaction.asset_ticker):
                    
                    self.add_asset(
                        transaction.asset_ticker,
                        transaction.asset_name,
                        transaction.asset_type
                        )

                    if log_process:
                        print("Create -", transaction.asset_ticker)
            

                # Compra de ativo
                if transaction.flow == "Entrada":

                    asset = self.find_asset(transaction.asset_ticker)

                    asset.buy(
                        transaction.quantity,
                        transaction.value
                        )
                    
                    if log_process:
                        print("Buy -", transaction.asset_ticker)


                # Venda de ativo
                if transaction.flow == "Saida":

                    asset = self.find_asset(transaction.asset_ticker)

                    asset.sell(
                        transaction.quantity,
                        transaction.value
                        )

            # Desdobro do ativo
            elif transaction.action == "Desdobro":

                asset = self.find_asset(transaction.asset_ticker)

                asset.split(transaction.quantity)

                if log_process:
                    print("Split -", transaction.asset_ticker)

            
            # Recebimento de divendendo ou rendimento
            elif transaction.action in ["Dividendo", "Rendimento"]:

                asset = self.find_asset(transaction.asset_ticker)

                asset.income_distribution(transaction.value)

                if log_process:
                    print("Income -", transaction.asset_ticker)           

            else:
                continue


## Test - Wallet

In [183]:
wallet = Wallet('my wallet')

In [156]:
wallet.add_asset('TEST12', 'Nome do produto', 'TIPO', log=True)
wallet.print_assets()

TEST12 added
Ticker: TEST12
Nome do produto: Nome do produto
Tipo: TIPO
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00
------------------------------------------------------------


In [184]:
wallet.track(transactions, log_process=True)

Create - BIDI4
Buy - BIDI4
Create - MGLU3
Buy - MGLU3
Create - VVAR3
Buy - VVAR3
Create - SPTW11
Buy - SPTW11
Income - SPTW11
Create - XPML11
Buy - XPML11
Buy - SPTW11
Create - BTOW3
Buy - BTOW3
Buy - VVAR3
Income - XPML11
Buy - BTOW3
Buy - VVAR3
Buy - MGLU3
Income - SPTW11
Buy - MGLU3
Income - XPML11
Buy - MGLU3
Create - JHSF3
Buy - JHSF3
Buy - MGLU3
Buy - VVAR3
Buy - MGLU3
Income - SPTW11
Buy - VVAR3
Create - AZUL4
Buy - AZUL4
Buy - VVAR3
Buy - MGLU3
Income - SPTW11
Buy - MGLU3
Create - PETR3
Buy - PETR3
Income - SPTW11
Create - WEGE3
Buy - WEGE3
Buy - VVAR3
Create - ITUB4
Buy - ITUB4
Create - CYRE3
Buy - CYRE3
Income - SPTW11
Income - XPML11
Income - SPTW11
Buy - CYRE3
Buy - SPTW11
Income - XPML11
Buy - ITUB4
Buy - CYRE3
Buy - MGLU3
Buy - ITUB4
Income - ITUB4
Create - B3SA3
Buy - B3SA3
Create - VALE3
Buy - VALE3
Income - SPTW11
Income - WEGE3
Income - MGLU3
Buy - VVAR3
Income - ITUB4
Create - BBDC3
Buy - BBDC3
Buy - SPTW11
Income - SPTW11
Buy - MGLU3
Income - XPML11
Buy - B3SA3
Inco

In [188]:
wallet_position = wallet.print_wallet()
wallet_position.loc[wallet_position['QUANTITY'] != 0].sort_values(by=['TYPE', 'PRODUCT_NAME'])

,TICKER,TYPE,PRODUCT_NAME,QUANTITY,PRICE,VALUE,INCOME
12,B3SA3,ACAO,"B3 S.A. – BRASIL, BOLSA, BALCÃO",1002,15.51,15537.88,685.28
1,MGLU3,ACAO,MAGAZINE LUIZA S.A.,700,22.51,15756.83,6.12
22,PETR4,ACAO,PETROLEO BRASILEIRO S.A. PETROBRAS,86,23.18,1993.57,648.63
8,PETR3,ACAO,PETROLEO BRASILEIRO S/A PETROBRAS,100,19.10,1910.00,778.08
13,VALE3,ACAO,VALE S.A.,20,61.09,1221.80,378.81
9,WEGE3,ACAO,WEG S.A.,20,21.11,422.10,12.01
20,AMZO34,BDR,"AMAZON.COM, INC",340,5.72,1943.95,0
27,NFLX34,BDR,"NETFLIX, INC.",46,19.94,917.24,0
26,NUBR33,BDR,NU HOLDINGS LTD.,598,8.36,4999.28,0
19,NVDC34,BDR,NVIDIA CORP,120,14.65,1758.00,1.78


In [ ]:
wallet.add_asset('TEST12', 'Nome do produto', 'TIPO', log=True)

TEST12 added


In [ ]:
wallet.asset_exist("TEST12")

True

In [ ]:
wallet.find_asset("TEST12")

In [ ]:
wallet.count_assets()

1

## Tracker

In [87]:
transactions.data.columns

Index(['FLOW', 'DATE', 'ACTION_TYPE', 'ASSET_TYPE', 'TICKER', 'PRODUCT_NAME',
       'BROKER', 'QUANTITY', 'PRICE_UNIT', 'VALUE'],
      dtype='object')

In [88]:
transactions.data.head()

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
0,Entrada,2019-12-31,Transferência - Liquidação,ACAO,BIDI4,BANCO INTER S.A.,EASYNVEST - TITULO CV S/A,104,15.7,1632.8
1,Entrada,2019-12-31,Transferência - Liquidação,ACAO,MGLU3,MAGAZINE LUIZA S.A.,EASYNVEST - TITULO CV S/A,33,49.37,1629.21
2,Entrada,2019-12-31,Transferência - Liquidação,ACAO,VVAR3,VIA VAREJO S.A.,EASYNVEST - TITULO CV S/A,130,11.73,1524.9
3,Entrada,2019-12-31,Transferência - Liquidação,FII,SPTW11,SP DOWNTOWN FDO INV IMOB FII,EASYNVEST - TITULO CV S/A,2,115.99,231.98
4,Saida,2020-01-02,Juros Sobre Capital Próprio - Transferido,ACAO,BIDI4,BANCO INTER S/A,NU INVEST CORRETORA DE VALORES S.A.,104,0.02,1.6


In [ ]:
transactions.data

In [103]:
def track(transactions, print_process=False, print_transaction=False):

    for index, row in transactions.data.iterrows():

        transaction = Transaction(
            flow = row['FLOW'],
            date = row['DATE'],
            action = row['ACTION_TYPE'],
            asset_type = row['ASSET_TYPE'],
            asset_ticker = row['TICKER'],
            asset_name = row['PRODUCT_NAME'], 
            broker = row['BROKER'],
            quantity = row['QUANTITY'],
            price = row['PRICE_UNIT'],
            value = row['VALUE']
            )

        if print_transaction:
            transaction.print_transaction()

        if transaction.action == "Transferência - Liquidação":

            if not wallet.asset_exist(transaction.asset_ticker):
                
                wallet.add_asset(
                    transaction.asset_ticker,
                    transaction.asset_name,
                    transaction.asset_type
                    )

                if print_process:
                    print("Create -", transaction.asset_ticker)
                


        continue

        if transaction.  row["ACTION_TYPE"] == "Transferência - Liquidação":
        
            # Avalia se o ativo existe na wallet, se não, cria o ativo
            if not wallet.asset_exist(row['TICKER']):
                print('cria ativo', row["TICKER"])
                wallet.add_asset(row['TICKER'], row['PRODUCT_NAME'], row['ASSET_TYPE'], log=imprime)

            if row['FLOW'] == "Entrada":
                asset = wallet.find_asset(row["TICKER"])
                asset.buy(row["QUANTITY"], row["VALUE"])

                print("comprou", row["TICKER"])


            if row['FLOW'] == "Saida":
                asset = wallet.find_asset(row["TICKER"])
                asset.sell(row["QUANTITY"], row["VALUE"])

                print("vendeu", row["TICKER"])
            
        elif row["ACTION_TYPE"] == "Desdobro":
            asset = wallet.find_asset(row["TICKER"])    
            asset.split(row["QUANTITY"])
            print("desdobrou", row["TICKER"])
            
        elif row["ACTION_TYPE"] in ["Dividendo", "Rendimento"]:
            asset = wallet.find_asset(row["TICKER"])
            print(asset.income)
            asset.income_distribution(row["VALUE"])
            print("DIVIDENDO")
        

        else:
            continue

In [104]:
track(transactions, print_process=True)

In [107]:
wallet.print_assets()

Ticker: TEST12
Nome do produto: Nome do produto
Tipo: TIPO
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00
------------------------------------------------------------
Ticker: BIDI4
Nome do produto: BANCO INTER S.A.
Tipo: ACAO
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00
------------------------------------------------------------
Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A.
Tipo: ACAO
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00
------------------------------------------------------------
Ticker: VVAR3
Nome do produto: VIA VAREJO S.A.
Tipo: ACAO
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00
------------------------------------------------------------
Ticker: SPTW11
Nome do produto: SP DOWNTOWN FDO INV IMOB FII
Tipo: FII
Quantidade: 0
Valor: 0
Preço médio: 0.00
Rendimentos: 0.00
------------------------------------------------------------
Ticker: XPML11
Nome do produto: XP MALLS FDO INV IMOB FII
Tipo: FII
Quantidade: 0
Valor: 0


In [62]:
imprime = True

# create wallet
wallet = Wallet("wallet Teste")

for index, row in transactions.data.iterrows():
    
    if imprime:
        print(row['DATE'], row['TICKER'], row["QUANTITY"], row["VALUE"], row['FLOW'], row['ACTION_TYPE'])
    
    if row["ACTION_TYPE"] == "Transferência - Liquidação":
    
        # Avalia se o ativo existe na wallet, se não, cria o ativo
        if not wallet.asset_exist(row['TICKER']):
            print('cria ativo', row["TICKER"])
            wallet.add_asset(row['TICKER'], row['PRODUCT_NAME'], row['ASSET_TYPE'], log=imprime)

        if row['FLOW'] == "Entrada":
            asset = wallet.find_asset(row["TICKER"])
            asset.buy(row["QUANTITY"], row["VALUE"])

            print("comprou", row["TICKER"])


        if row['FLOW'] == "Saida":
            asset = wallet.find_asset(row["TICKER"])
            asset.sell(row["QUANTITY"], row["VALUE"])

            print("vendeu", row["TICKER"])
        
    elif row["ACTION_TYPE"] == "Desdobro":
        asset = wallet.find_asset(row["TICKER"])
        asset.split(row["QUANTITY"])
        print("desdobrou", row["TICKER"])
        
    elif row["ACTION_TYPE"] in ["Dividendo", "Rendimento"]:
        asset = wallet.find_asset(row["TICKER"])
        print(asset.income)
        asset.income_distribution(row["VALUE"])
        print("DIVIDENDO")
    

    else:
        continue
    
    

2019-12-31 00:00:00 BIDI4 104 1632.8 Entrada Transferência - Liquidação
cria ativo BIDI4
BIDI4 added
comprou BIDI4
2019-12-31 00:00:00 MGLU3 33 1629.21 Entrada Transferência - Liquidação
cria ativo MGLU3
MGLU3 added
comprou MGLU3
2019-12-31 00:00:00 VVAR3 130 1524.9 Entrada Transferência - Liquidação
cria ativo VVAR3
VVAR3 added
comprou VVAR3
2019-12-31 00:00:00 SPTW11 2 231.98 Entrada Transferência - Liquidação
cria ativo SPTW11
SPTW11 added
comprou SPTW11
2020-01-02 00:00:00 BIDI4 104 1.6 Saida Juros Sobre Capital Próprio - Transferido
2020-01-02 00:00:00 BIDI4 104 1.6 Entrada Juros Sobre Capital Próprio - Transferido
2020-01-03 00:00:00 MGLU3 33 1.01 Saida Juros Sobre Capital Próprio - Transferido
2020-01-03 00:00:00 MGLU3 33 1.01 Entrada Juros Sobre Capital Próprio - Transferido
2020-01-08 00:00:00 BIDI4 104 1.6 Entrada Juros Sobre Capital Próprio
2020-01-09 00:00:00 SPTW11 2 1.1 Entrada Amortização
2020-01-09 00:00:00 SPTW11 2 1.46 Entrada Rendimento
0
DIVIDENDO
2020-01-10 00:00:0

In [63]:
wallet.print_assets()

Ticker: BIDI4
Nome do produto: BANCO INTER S.A.
Tipo: ACAO
Quantidade: 0
Valor: 1632.8
Preço médio: 408.20
Rendimentos: 0.00
------------------------------------------------------------
Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A.
Tipo: ACAO
Quantidade: 700
Valor: 15756.830000000002
Preço médio: 22.51
Rendimentos: 6.12
------------------------------------------------------------
Ticker: VVAR3
Nome do produto: VIA VAREJO S.A.
Tipo: ACAO
Quantidade: 0
Valor: 6900.139999999999
Preço médio: 27.71
Rendimentos: 0.00
------------------------------------------------------------
Ticker: SPTW11
Nome do produto: SP DOWNTOWN FDO INV IMOB FII
Tipo: FII
Quantidade: 0
Valor: 1687.87
Preço médio: 84.39
Rendimentos: 328.65
------------------------------------------------------------
Ticker: XPML11
Nome do produto: XP MALLS FDO INV IMOB FII
Tipo: FII
Quantidade: 0
Valor: 707.4
Preço médio: 141.48
Rendimentos: 11.17
------------------------------------------------------------
Ticker: BTOW3
Nome do 

In [21]:
transactions.data[transactions.data.TICKER == "VALE3"]

,FLOW,DATE,ACTION_TYPE,ASSET_TYPE,TICKER,PRODUCT_NAME,BROKER,QUANTITY,PRICE_UNIT,VALUE
74,Entrada,2020-08-05,Transferência - Liquidação,ACAO,VALE3,VALE S.A.,NU INVEST CORRETORA DE VALORES S.A.,20,61.09,1221.8
97,Entrada,2020-09-30,Dividendo,ACAO,VALE3,VALE S.A.,EASYNVEST - TITULO CV S/A,20,-,-
98,Entrada,2020-09-30,Juros Sobre Capital Próprio,ACAO,VALE3,VALE S.A.,EASYNVEST - TITULO CV S/A,20,-,-
156,Entrada,2021-03-15,Dividendo,ACAO,VALE3,VALE S.A.,EASYNVEST - TITULO CV S/A,20,3.43,68.51
157,Entrada,2021-03-15,Juros Sobre Capital Próprio,ACAO,VALE3,VALE S.A.,EASYNVEST - TITULO CV S/A,20,0.84,14.21
186,Entrada,2021-06-30,Dividendo,ACAO,VALE3,VALE S.A.,EASYNVEST - TITULO CV S/A,20,0.72,14.4
187,Entrada,2021-06-30,Dividendo,ACAO,VALE3,VALE S.A.,EASYNVEST - TITULO CV S/A,20,1.47,29.38
214,Entrada,2021-09-30,Dividendo,ACAO,VALE3,VALE S.A.,NU INVEST CORRETORA DE VALORES S.A.,20,8.2,163.94
245,Entrada,2022-03-16,Dividendo,ACAO,VALE3,VALE S.A.,NU INVEST CORRETORA DE VALORES S.A.,20,3.72,74.38


In [102]:
wallet.print_assets()

Ticker: BIDI4
Nome do produto: BANCO INTER S.A. (BIDI4)
Tipo: ACAO
Quantidade: 0
Valor: 1632.8
Preço médio: 408.20
Rendimentos: 0.00
------------------------------------------------------------
Ticker: MGLU3
Nome do produto: MAGAZINE LUIZA S.A. (MGLU3)
Tipo: ACAO
Quantidade: 700
Valor: 15756.940000000002
Preço médio: 22.51
Rendimentos: 6.12
------------------------------------------------------------
Ticker: VVAR3
Nome do produto: VIA VAREJO S.A. (VVAR3)
Tipo: ACAO
Quantidade: 0
Valor: 6900.659999999999
Preço médio: 27.71
Rendimentos: 0.00
------------------------------------------------------------
Ticker: SPTW11
Nome do produto: SP DOWNTOWN FDO INV IMOB FII (SPTW11)
Tipo: FII
Quantidade: 0
Valor: 1687.88
Preço médio: 84.39
Rendimentos: 328.65
------------------------------------------------------------
Ticker: XPML11
Nome do produto: XP MALLS FDO INV IMOB FII (XPML11)
Tipo: FII
Quantidade: 0
Valor: 707.4
Preço médio: 141.48
Rendimentos: 11.17
-----------------------------------------